In [1]:
import ee
from IPython import display
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from osgeo import gdal
import tempfile
import urllib
import zipfile
from mpl_toolkits.basemap import Basemap
import numpy as np
import datetime
import pandas as pd

In [2]:
ee.Initialize()

# VETTORI 1EGLF7Gh-5Z3Zt9pxdKF-jBDtxQ-DnpJzz9D1BGQJ
countries_gaul = ee.FeatureCollection('ft:1ZDEMjtnWm_smu7l_z3fx91BbxyCRzP2A3cEMrEiP')

# AREA OF INTEREST
region_L1 = ee.Geometry.Polygon([[[-30, -40], [65, -30], [65, 40], [-30, 40]]])
region_json= [[-30, -40], [65, -30], [65, 40], [-30, 40]]

subregions = {
  'na': 'Northern Africa',
  'ma': 'Middle Africa',
  'wa': 'Western Africa', 
  'ea': 'Eastern Africa',
  'sa': 'Southern Africa', 
  'ws': 'Western Asia'
}

In [ ]:
subregion = 'ws'
year = 2010
name_asset = 'T_Annual'
selected_subregion = subregions[subregion]
filtered = countries_gaul.filter(ee.Filter.eq('subregion', selected_subregion))
country_number = filtered.size().getInfo()
print(country_number)
#for indice in range(0,country_number):
#    print(indice,filtered.getInfo()['features'][indice]['properties']['name'])
radice = 'projects/fao-wapor/'
raster_name = radice + name_asset + "/" + name_asset + "-" + str(year)
print(raster_name)
prefix = raster_name.split("/")[2]
raster_stats = ee.Image(raster_name).toFloat()
scale_calc = ee.Image(raster_stats).projection().nominalScale().getInfo()
print('Image scale: ', scale_calc)

In [142]:
means = raster_stats.reduceRegions(
    collection = filtered,
    reducer = ee.Reducer.mean(),
    scale = scale_calc,
)

reducers = ee.Reducer.minMax().combine(
    reducer2 = ee.Reducer.sum(),
    sharedInputs = True
);

minMaxStds = raster_stats.reduceRegions(
            collection = filtered,
            reducer = reducers,
            scale = scale_calc,
);

In [143]:
features_m = means.getInfo()['features']
features_mms = minMaxStds.getInfo()['features']
df_m = pd.DataFrame(data=features_m[0:], columns = features_m[0])
df_mms = pd.DataFrame(data=features_mms[0:], columns = features_mms[0])
serie_m = df_m['properties'].apply(pd.Series)
serie_mms = df_mms['properties'].apply(pd.Series)
df_m = pd.DataFrame(serie_m[['fid', 'gaul_code', 'iso3', 'mean', 'name', 'region','subregion']])
df_mms = pd.DataFrame(serie_mms[['fid', 'min', 'max','sum']])
df_stats = df_m.set_index('fid').join(df_mms.set_index('fid'), lsuffix='_df_m', rsuffix='_df_mms')
df_stats

,gaul_code,iso3,mean,name,region,subregion,min,max,sum
fid,,,,,,,,,
140.0,142.0,LSO,244.489958,Lesotho,Africa,Southern Africa,0.0,1227.203979,1.391475e+08
35.0,35.0,BWA,506.877511,Botswana,Africa,Southern Africa,0.0,2394.580078,5.154604e+09
169.0,172.0,NAM,278.069546,Namibia,Africa,Southern Africa,0.0,2247.160889,4.031718e+09
223.0,227.0,ZAF,328.322407,South Africa,Africa,Southern Africa,0.0,2420.906006,7.450067e+09
230.0,235.0,SWZ,676.646543,Swaziland,Africa,Southern Africa,0.0,1742.990967,2.127160e+08


In [144]:
file_name = name_asset + "_" + str(year) + "_" + subregion # + prefix.lower()
file_name
df_stats.to_csv('stats/' + file_name + '.csv')